In [ ]:
import numpy as np
import cv2
pitcher_name = 'MichaelWacha'  # Replace with the actual pitcher name
video_path =  'C:\\Users\\devae\\OneDrive\\Desktop\\MotionAGFormer\\test_vid\\'
print(video_path+pitcher_name+'.mp4')
# Open the video file
cap = cv2.VideoCapture(video_path+pitcher_name+'.mp4')

# Check if video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
else:
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    # Convert frames to a numpy array
    video_array = np.array(frames)
    print(f"Video loaded successfully. Shape: {video_array.shape}")

    # Release the video capture object
    cap.release()

# Load the keypoints file
try:
    keypoints_array = np.load(video_path+pitcher_name+'.npz')
    # Assuming keypoints.npz contains a single array, access it like this:
    # keypoints_data = keypoints_array['arr_0'] # Or the correct key if known
    print("Keypoints file loaded successfully.") # Add more specific message if key is known
except FileNotFoundError:
    print("Error: keypoints.npz not found.")
except Exception as e:
    print(f"An error occurred while loading keypoints: {e}")

# You now have video_array and potentially keypoints_data (depending on the structure of keypoints.npz)
# You can perform further operations with these arrays.

C:\Users\devae\OneDrive\Desktop\MotionAGFormer\test_vid\MichaelWacha.mp4
Video loaded successfully. Shape: (139, 528, 570, 3)
Keypoints file loaded successfully.


In [40]:
keypoints = keypoints_array['reconstruction']

In [41]:
keypoints.shape

(1, 139, 17, 3)

In [42]:
import cv2
import numpy as np

class PointEditor:
    def __init__(self, video, points_3d, scale=2.0, radius=3):
        """
        Args:
            video: numpy array of shape (num_frames, H, W, 3)
            points_3d: numpy array of shape (num_frames, num_points, 3) => (x, y, confidence)
            scale: float, how much to scale up the displayed frame
            radius: int, radius of the keypoints displayed
        """
        self.video = video
        self.points = points_3d.copy()
        self.original_points = points_3d.copy()
        self.edited_flags = np.any(self.points[..., :2] != self.original_points[..., :2], axis=-1)
        self.current_frame = 0
        self.dragging_point = None
        self.radius = radius
        self.scale = scale

        # Define 17 distinct colors (BGR)
        self.point_colors = [
            (0, 0, 255),       # 0 - Red
            (0, 128, 255),     # 1 - Orange
            (0, 255, 255),     # 2 - Yellow
            (0, 255, 0),       # 3 - Green
            (255, 255, 0),     # 4 - Cyan
            (255, 0, 0),       # 5 - Blue
            (255, 0, 255),     # 6 - Purple
            (128, 0, 128),     # 7 - Violet
            (128, 128, 0),     # 8 - Olive
            (0, 128, 128),     # 9 - Teal
            (128, 0, 0),       # 10 - Maroon
            (0, 0, 128),       # 11 - Navy
            (0, 255, 128),     # 12 - Light Green
            (255, 128, 0),     # 13 - Light Orange
            (255, 0, 128),     # 14 - Pinkish
            (128, 255, 0),     # 15 - Lime
            (128, 128, 255),   # 16 - Light Blue
        ]

        # Define lines as pairs of point indices
        self.lines = [
            (5, 6),  # Blue and Purple
            (2, 3),  # Yellow and Green
            (1, 2),  # Orange and Yellow
            (4, 5),  # Cyan and Blue
            (15,16),  # Lime and Light Blue
            (14,15),
            (12,13),  # Light Green and Light Orange
            (11,12)
        ]

    def edit(self):
        cv2.namedWindow("Frame")
        cv2.setMouseCallback("Frame", self.mouse_event)

        while self.current_frame < len(self.video):
            self.frame = self.video[self.current_frame].copy()
            frame_points = self.points[self.current_frame]

            while True:
                display = self.frame.copy()

                # Draw lines between specified point pairs
                for i, j in self.lines:
                    x1, y1, c1 = frame_points[i]
                    x2, y2, c2 = frame_points[j]
                    pt1 = (int(x1), int(y1))
                    pt2 = (int(x2), int(y2))
                    cv2.line(display, pt1, pt2, (255, 255, 255), 1)  # White line

                # Draw points
                for idx, pt in enumerate(frame_points):
                    x, y, conf = pt
                    color = self.point_colors[idx % len(self.point_colors)]
                    cv2.circle(display, (int(x), int(y)), self.radius, color, -1)

                # Resize the frame
                display_resized = cv2.resize(display, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_LINEAR)

                cv2.putText(display_resized,
                            f'Frame {self.current_frame+1}/{len(self.video)} - "n": next, "esc": exit',
                            (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)

                cv2.imshow("Frame", display_resized)
                key = cv2.waitKey(20) & 0xFF

                if key == ord('n'):
                    break
                elif key == 27:
                    cv2.destroyAllWindows()
                    return self._finalize()

            self.current_frame += 1

        cv2.destroyAllWindows()
        return self._finalize()

    def _finalize(self):
        self.points[self.edited_flags, 2] = 1.0
        return self.points

    def mouse_event(self, event, x, y, flags, param):
        x = int(x / self.scale)
        y = int(y / self.scale)

        frame_points = self.points[self.current_frame]

        if event == cv2.EVENT_LBUTTONDOWN:
            for idx, pt in enumerate(frame_points):
                px, py, _ = pt
                if np.linalg.norm(np.array([px, py]) - np.array([x, y])) < self.radius * 2:
                    self.dragging_point = idx
                    break

        elif event == cv2.EVENT_MOUSEMOVE and self.dragging_point is not None:
            self.points[self.current_frame, self.dragging_point, 0] = x
            self.points[self.current_frame, self.dragging_point, 1] = y
            self.edited_flags[self.current_frame, self.dragging_point] = True

        elif event == cv2.EVENT_LBUTTONUP:
            self.dragging_point = None



In [43]:
import numpy as np

# Load your .npz file
video = video_array
updated_points = keypoints[0]

In [44]:

# Edit
editor = PointEditor(video, updated_points)
updated_points = editor.edit()

# Save if needed
np.savez(video_path+pitcher_name+"_fixed.npz", points=updated_points, video=video)
np.save(video_path+pitcher_name+"_fixed.npy", updated_points)
